![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [61]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [62]:
# Start coding here
# Use as many cells as you need

# Code for Datacamp Project: "Analyizing Crime In Los Angeles"

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Fun: explore_df
def explore_df(df, method):
    """
    Function to run describe, head, or info on df.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to explore.
    method : {'desc', 'head', 'info', 'all'}
        Specify the method to use.
        - 'desc': Display summary statistics using describe().
        - 'head': Display the first few rows using head().
        - 'info': Display concise information about the DataFrame using info().
        - 'na': Display counts of NAs per column and percentage of NAs per column.
        - 'all': Display all information from above options.

    Returns
    -------
    None
    """
    if method.lower() == "desc":
        print(df.describe())
    elif method.lower() == "head":
        pd.set_option('display.max_columns', None)
        print(df.head())
        pd.reset_option('display.max_columns')
    elif method.lower() == "info":
        print(df.info())
    elif method.lower() == "na":
        print(f"\n\n<<______NA_COUNT______>>")
        print(df.isna().sum())
        print(f"\n\n<<______NA_PERCENT______>>")
        print((df.isna().sum() / df.shape[0])*100)
    elif method.lower() == "all":
        print("<<______HEAD______>>")
        pd.set_option('display.max_columns', None)
        print(df.head())
        pd.reset_option('display.max_columns')
        print(f"\n\n<<______DESCRIBE______>>")
        print(df.describe())
        print(f"\n\n<<______INFO______>>")
        print(df.info())
        print(f"\n\n<<______NA_COUNT______>>")
        print(df.isna().sum())
        print(f"\n\n<<______NA_PERCENT______>>")
        print((df.isna().sum() / df.shape[0])*100)
    else:
        print("Methods: 'desc', 'head', 'info' or 'all'")

# Import data and explore
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})

explore_df(crimes, "all")



# Question 1. Which hour has the highest frequency of crimes? Store as an integer variable called peak_crime_hour.

# Add ':' between the hour and minute in TIME OCC
crimes['TIME OCC'] = (
    crimes['TIME OCC']
    .str.slice_replace(2, 0, ':') # add : between H and M
    +':00' # add seconds at the end (useful for timedelta)
)

# sanity check
print(crimes['TIME OCC'][0])

# Make TIME OCC a timedelta object 
crimes['TIME OCC'] = pd.to_timedelta(crimes['TIME OCC']+'')

# Create new column: DATETIME_OCC
# Combination of TIME OCC and DATE OCC as one datetime
crimes['DATETIME_OCC'] = crimes['DATE OCC'] + crimes['TIME OCC']

# sanity check
print(crimes['DATETIME_OCC'].dtype, crimes['DATETIME_OCC'][0])

# crime frequency per hour
crime_freq_per_hour = crimes['DATETIME_OCC'].dt.hour.value_counts()

# Question 1. Answer
peak_crime_hour = 12



# Question 2. Which area has the largest frequency of night crimes (crimes committed between 10pm and 3:59am)? Save as a string variable called peak_night_crime_location.

# create new df: crimes_2200_to_0359
# df containing crimes only between those hours

# filter from 22:00 to 23:59
filter_2200_to_2359 = (
    (crimes['DATETIME_OCC'].dt.hour >= 22) &
    (crimes['DATETIME_OCC'].dt.hour <= 23)
)

# filter from 00:00 to 03:59
filter_0000_to_0359 = (
    (crimes['DATETIME_OCC'].dt.hour >= 0) &
    (crimes['DATETIME_OCC'].dt.hour <= 3)
)

# combined time filter
time_filter = filter_2200_to_2359 | filter_0000_to_0359

# create new df: crimes_2200_to_0359
crimes_2200_to_0359 = crimes[time_filter]

# sanity checks (1-3)
# (1) check individual filters
print("Rows between 10pm and 11:59pm:")
print(crimes[filter_2200_to_2359].head())
print("\nRows between 12am and 3:59am:")
print(crimes[filter_0000_to_0359].head())

# (2) verify combined time filter
print("\nCombined time filter:")
print(crimes[time_filter].head())

# (3) inspect resulting df
print("\nResulting DataFrame (crimes_2200_to_0359):")
print(crimes_2200_to_0359.head())

# Question 2. Answer
peak_night_crime_location = (
    crimes_2200_to_0359['AREA NAME']
    .value_counts() # count crimes/rows per area name
    .idxmax() # get id/name of highest value
)




# Question 3. Identify the number of crimes committed against victims by age group (0-17, 18-25, 26-34, 35-44, 45-54, 55-64, 65+). Save as a pandas Series called victim_ages.

# create new column: AGE_GROUP (1-2)
# (1) age group bins and labels for pd.cut
age_bins = [0,17,25,34,44,54,64,float('inf')]
age_labels = [
    '0-17', 
    '18-25', 
    '26-34', 
    '35-44', 
    '45-54', 
    '55-64', 
    '65+'
]

# (2) use pd.cut to create AGE_GROUP
crimes["AGE_GROUP"] = pd.cut(
    crimes["Vict Age"],
    bins=age_bins,
    labels=age_labels
)

# sanity check: there is a NaN further investigate
print(crimes["AGE_GROUP"].unique())

# further investigation shows
# that there are ages 0, -1 and -2 in the df
print(crimes[crimes["AGE_GROUP"].isna()]["Vict Age"].unique())

# Question 3. Answer
victim_ages = (
    crimes["AGE_GROUP"]
    .value_counts() # count crimes per age group
    .sort_index() # sort by age group/index
)

# sanity check
print(victim_ages)

<<______HEAD______>>
       DR_NO  Date Rptd   DATE OCC TIME OCC    AREA NAME        Crm Cd Desc  \
0  220314085 2022-07-22 2020-05-12     1110    Southwest  THEFT OF IDENTITY   
1  222013040 2022-08-06 2020-06-04     1620      Olympic  THEFT OF IDENTITY   
2  220614831 2022-08-18 2020-08-17     1200    Hollywood  THEFT OF IDENTITY   
3  231207725 2023-02-27 2020-01-27     0635  77th Street  THEFT OF IDENTITY   
4  220213256 2022-07-14 2020-07-14     0900      Rampart  THEFT OF IDENTITY   

   Vict Age Vict Sex Vict Descent Weapon Desc  Status Desc  \
0        27        F            B         NaN  Invest Cont   
1        60        M            H         NaN  Invest Cont   
2        28        M            H         NaN  Invest Cont   
3        37        M            H         NaN  Invest Cont   
4        79        M            B         NaN  Invest Cont   

                                  LOCATION  
0  2500 S  SYCAMORE                     AV  
1  3300    SAN MARINO                   S